In [1]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import VGG16, MobileNet

# Define ensemble model using VGG16 and MobileNet
def ensemble_model(input_shape=(224, 224, 3), num_classes=6):
    # Load pre-trained VGG16 model
    vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in vgg_model.layers:
        layer.trainable = False

    # Load pre-trained MobileNet model
    mobilenet_model = MobileNet(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in mobilenet_model.layers:
        layer.trainable = False

    # Define input layer
    input_layer = layers.Input(shape=input_shape)

    # Extract features using VGG16
    vgg_features = vgg_model(input_layer)
    vgg_features = layers.GlobalAveragePooling2D()(vgg_features)

    # Extract features using MobileNet
    mobilenet_features = mobilenet_model(input_layer)
    mobilenet_features = layers.GlobalAveragePooling2D()(mobilenet_features)

    # Concatenate features
    merged_features = layers.Concatenate()([vgg_features, mobilenet_features])

    # Add a fully connected layer
    merged_features = layers.Dense(512, activation='relu')(merged_features)

    # Output layer
    output = layers.Dense(num_classes, activation='softmax')(merged_features)

    # Create model
    model = Model(inputs=input_layer, outputs=output)

    return model

# Define data directories
# Define data directories
train_dir = 'D:\P_126\data_idumfc\preprocessed_train'
val_dir = 'D:\P_126\data_idumfc\preprocessed_val'
test_dir = 'D:\P_126\data_idumfc\preprocessed_test'
# Define image size and batch size
image_size = (224, 224)
batch_size = 32

# Create data generators with data augmentation for training and validation
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Define number of classes
num_classes = len(train_generator.class_indices)

# Load ensemble model
model = ensemble_model(input_shape=image_size + (3,), num_classes=num_classes)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=150,
    epochs=100,
    validation_data=val_generator,
    validation_steps=len(val_generator)
)

# Evaluate the model
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Accuracy:", test_accuracy)





Found 16050 images belonging to 6 classes.
Found 4104 images belonging to 6 classes.



Epoch 1/100


150/150 [==============================] - 1217s 8s/step - loss: 0.9944 - accuracy: 0.6423 - val_loss: 0.7728 - val_accuracy: 0.7188
Epoch 2/100
150/150 [==============================] - 1207s 8s/step - loss: 0.6757 - accuracy: 0.7598 - val_loss: 0.6229 - val_accuracy: 0.7654
Epoch 3/100
150/150 [==============================] - 1215s 8s/step - loss: 0.6317 - accuracy: 0.7783 - val_loss: 0.6182 - val_accuracy: 0.7770
Epoch 4/100
150/150 [==============================] - 1191s 8s/step - loss: 0.5895 - accuracy: 0.7790 - val_loss: 0.5690 - val_accuracy: 0.7851
Epoch 5/100
150/150 [==============================] - 1194s 8s/step - loss: 0.5601 - accuracy: 0.7940 - val_loss: 0.5229 - val_accuracy: 0.8087
Epoch 6/100
150/150 [==============================] - 1191s 8s/step - loss: 0.5390 - accuracy: 0.8048 - val_loss: 0.5591 - val_accuracy: 0.7936
Epoch 7/100
150/150 [==================

In [2]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

# Predict probabilities for test data
Y_pred = model.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)
class_labels = list(test_generator.class_indices.keys())

# Get true labels
true_labels = test_generator.classes

# Print classification report
print("Classification Report:")
print(classification_report(true_labels, y_pred, target_names=class_labels))

# Calculate confusion matrix
conf_matrix = confusion_matrix(true_labels, y_pred)

# Print confusion matrix
print("Confusion Matrix:")
print(conf_matrix)

129/129 [==============================] - 522s 4s/step
Classification Report:
              precision    recall  f1-score   support

CARDIOMEGALY       0.15      0.13      0.14       685
       COVID       0.12      0.11      0.11       685
      NORMAL       0.19      0.20      0.19       685
   PNEUMONIA       0.15      0.13      0.14       685
PNEUMOTHORAX       0.14      0.18      0.16       685
TUBERCULOSIS       0.14      0.15      0.15       685

    accuracy                           0.15      4110
   macro avg       0.15      0.15      0.15      4110
weighted avg       0.15      0.15      0.15      4110

Confusion Matrix:
[[ 90 116 125  81 133 140]
 [115  72 114 107 155 122]
 [ 93 100 136  96 140 120]
 [112 110 117  90 147 109]
 [ 97 114 116 116 120 122]
 [101 107 107 109 159 102]]


In [3]:
# Save the model
model.save("vggmobilenet_cls600.h5")

C:\ProgramData\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
